In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/finetuned-with-mixup-and-cutmix/pytorch/default/1/best_model_finetuned (3).pth
/kaggle/input/finetuned-model-91/pytorch/default/1/best_model_finetuned (2).pth
/kaggle/input/without-finetuning/pytorch/default/1/best_model (12).pth
/kaggle/input/latest_finetune/pytorch/default/1/best_model_finetuned.pth
/kaggle/input/retesting_previous_with_contrastedge-in-tta/pytorch/default/1/best_model_finetuned (4).pth
/kaggle/input/finetuned_98/pytorch/default/1/best_model_finetuned (1).pth
/kaggle/input/best_model_without_finetune/pytorch/default/1/best_model (13).pth
/kaggle/input/deep-learning-spring-2025-project-1/cifar_test_nolabel.pkl
/kaggle/input/deep-learning-spring-2025-project-1/cifar-10-python/cifar-10-batches-py/data_batch_1
/kaggle/input/deep-learning-spring-2025-project-1/cifar-10-python/cifar-10-batches-py/data_batch_2
/kaggle/input/deep-learning-spring-2025-project-1/cifar-10-python/cifar-10-batches-py/batches.meta
/kaggle/input/deep-learning-spring-2025-project-1/cifa

In [4]:
import os
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as F 
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
!pip install torchsummary
import torchsummary
import random
from PIL import Image

import torch.nn.init as init

import torchvision
from torchvision import transforms

from torch.nn.utils import (
  parameters_to_vector as Params2Vec,
  vector_to_parameters as Vec2Params
)

from tqdm import tqdm
import torchvision.transforms.v2 as v2
import random
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.optim.lr_scheduler import LambdaLR

In [5]:
test_data_path = "/kaggle/input/deep-learning-spring-2025-project-1/cifar_test_nolabel.pkl"
with open(test_data_path, 'rb') as file:
    test_data = pickle.load(file, encoding='bytes')

# Extract test images and IDs
test_images = test_data[b'data']  # Shape (10000, 32, 32, 3) in numpy format
test_ids = test_data[b'ids']        # Test image IDs

print(f"Loaded test images shape: {test_images.shape}")

Loaded test images shape: (10000, 32, 32, 3)


In [6]:
def detect_weak_edges(image, threshold=50):
    """
    Detect weak edges using Sobel filters in PyTorch.
    
    Args:
        image (torch.Tensor): Input image tensor (C, H, W).
        threshold (int): Edge strength threshold to determine weak edges.

    Returns:
        bool: True if edges are weak, False otherwise.
    """
    # Convert RGB to grayscale using ITU-R BT.601 standard
    grayscale = 0.299 * image[0] + 0.587 * image[1] + 0.114 * image[2]  # Shape: (H, W)

    # Apply Sobel filters for edge detection
    sobel_x = torch.tensor([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]], dtype=torch.float32).unsqueeze(0).unsqueeze(0).to(image.device)
    sobel_y = torch.tensor([[-1, -2, -1], [0, 0, 0], [1, 2, 1]], dtype=torch.float32).unsqueeze(0).unsqueeze(0).to(image.device)

    # Reshape grayscale to (1, 1, H, W) for convolution
    grayscale = grayscale.unsqueeze(0).unsqueeze(0)  # Shape: (1, 1, H, W)

    # Apply Sobel convolution
    edge_x = F.conv2d(grayscale, sobel_x, padding=1)
    edge_y = F.conv2d(grayscale, sobel_y, padding=1)
    
    # Compute edge strength
    edges = torch.sqrt(edge_x ** 2 + edge_y ** 2)  # Edge magnitude
    edge_sum = edges.sum().item()  # Sum of edge intensities

    return edge_sum < threshold  # Return True if edges are weak

def enhance_contrast(image):
    """
    Enhance image contrast using PyTorch's histogram equalization.

    Args:
        image (torch.Tensor): Input image tensor (C, H, W).

    Returns:
        torch.Tensor: Contrast-enhanced image tensor (C, H, W).
    """
    # Convert to grayscale for histogram equalization
    grayscale = 0.299 * image[0] + 0.587 * image[1] + 0.114 * image[2]  # Shape: (H, W)

    # Compute histogram and cumulative distribution function (CDF)
    hist = torch.histc(grayscale, bins=256, min=0.0, max=1.0)
    cdf = hist.cumsum(0) / hist.sum()  # Normalize CDF

    # Apply histogram equalization
    equalized = torch.index_select(cdf, 0, (grayscale * 255).long().clamp(0, 255))  # Apply equalization
    equalized = equalized / equalized.max()  # Normalize

    # Scale RGB channels based on equalized grayscale
    enhanced_image = image * (equalized / (grayscale + 1e-6))  # Avoid division by zero
    enhanced_image = torch.clamp(enhanced_image, 0, 1)  # Ensure valid range

    return enhanced_image

class ContrastEnhancementTransform(torch.nn.Module):
    """
    Custom PyTorch transform to selectively enhance contrast for weak edge images.
    """
    def forward(self, img):
        """
        Apply contrast enhancement selectively based on weak edge detection.

        Args:
            img (torch.Tensor): Input image tensor (C, H, W).

        Returns:
            torch.Tensor: Transformed image tensor (C, H, W).
        """
        if detect_weak_edges(img):  # Apply only to weak edge images
            img = enhance_contrast(img)

        return img  # No conversion needed (still a PyTorch tensor)

In [ ]:
#Test time augmentation
tta_transforms = [
    # 1. Original image: Baseline without any augmentation.
    transforms.Compose([
        transforms.ToTensor(),
        ContrastEnhancementTransform(),  # Selectively enhance weak-edge images
        transforms.Normalize(mean=[0.49139968, 0.48215827, 0.44653124],
                             std=[0.24703233, 0.24348505, 0.26158768])
    ]),

    # 2. Horizontal Flip: Captures potential left-right symmetry.
    transforms.Compose([
        transforms.ToTensor(),
        ContrastEnhancementTransform(),  # Selectively enhance weak-edge images
        transforms.RandomHorizontalFlip(p=1.0),  # Always flip
        transforms.Normalize(mean=[0.49139968, 0.48215827, 0.44653124],
                             std=[0.24703233, 0.24348505, 0.26158768])
    ]),

    # 3. Fixed +5° Rotation: Accounts for slight rotation variations.
    transforms.Compose([
        transforms.ToTensor(),
        ContrastEnhancementTransform(),  # Selectively enhance weak-edge images
        transforms.RandomRotation(degrees=(5, 5)),
        transforms.Normalize(mean=[0.49139968, 0.48215827, 0.44653124],
                             std=[0.24703233, 0.24348505, 0.26158768])
    ]),

    # 4. Fixed -5° Rotation: Another slight rotation variant.
    transforms.Compose([
        transforms.ToTensor(),
        ContrastEnhancementTransform(),  # Selectively enhance weak-edge images
        transforms.RandomRotation(degrees=(-5, -5)),
        transforms.Normalize(mean=[0.49139968, 0.48215827, 0.44653124],
                             std=[0.24703233, 0.24348505, 0.26158768])
    ]),

    # 5. Center Crop Variant: Remove background bias by cropping and resizing.
    transforms.Compose([
        transforms.ToTensor(),
        ContrastEnhancementTransform(),  # Selectively enhance weak-edge images
        transforms.CenterCrop(28),  # Crop a central region
        transforms.Resize(32),      # Resize back to original size
        transforms.Normalize(mean=[0.49139968, 0.48215827, 0.44653124],
                             std=[0.24703233, 0.24348505, 0.26158768])
    ]),

    # 6. Mild Random Resized Crop: Subtle positional variation.
    transforms.Compose([
        transforms.ToTensor(),
        ContrastEnhancementTransform(),  # Selectively enhance weak-edge images
        transforms.RandomResizedCrop(32, scale=(0.9, 1.0)),
        transforms.Normalize(mean=[0.49139968, 0.48215827, 0.44653124],
                             std=[0.24703233, 0.24348505, 0.26158768])
    ]),

    # 7. Slight Brightness/Contrast Adjustment: Very mild color variation.
    transforms.Compose([
        transforms.ToTensor(),
        ContrastEnhancementTransform(),  # Selectively enhance weak-edge images
        transforms.ColorJitter(brightness=0.05, contrast=0.05),
        transforms.Normalize(mean=[0.49139968, 0.48215827, 0.44653124],
                             std=[0.24703233, 0.24348505, 0.26158768])
    ])
]

In [8]:
# Custom Test Dataset for CIFAR-10
class TestCIFAR10Dataset(Dataset):
    def __init__(self, images, ids, transform=None):
        self.images = images
        self.ids = ids
        self.transform = transform  # No transform applied during dataset init

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img = self.images[idx]
        # Convert numpy array to PIL image if necessary
        if isinstance(img, np.ndarray):
            # If image is channel-first, convert to channel-last
            if img.shape[0] == 3:
                img = np.transpose(img, (1, 2, 0))
            # Ensure image is in uint8 format
            if img.dtype != np.uint8:
                img = img.astype(np.uint8)
            img = Image.fromarray(img)
        # Apply transform if provided (for TTA, transform is applied later)
        if self.transform:
            img = self.transform(img)
        return img, self.ids[idx]

In [9]:
# Custom Collate Function
def custom_collate_fn(batch):
    images, ids = zip(*batch)
    # Return as lists so each image remains as a PIL image
    return list(images), list(ids)

In [10]:
# Create Dataset and DataLoader for Test Data
batch_size = 64   # Keeping Same as training and fine tuning
num_workers = 4   

final_dataset = TestCIFAR10Dataset(test_images, test_ids, transform=None)
final_loader = DataLoader(final_dataset,
                          batch_size=batch_size,
                          shuffle=False,
                          num_workers=num_workers,
                          collate_fn=custom_collate_fn)

In [11]:
def generate_final_predictions_csv(model, best_model_path, device):
    # Load best model weights and set model to evaluation mode
    model.load_state_dict(torch.load(best_model_path, map_location=device))
    model.to(device)
    model.eval()

    predictions = []
    with torch.no_grad():
        for images, ids in final_loader:
            batch_preds = []
            
            # Apply each TTA transform to the entire batch
            for tta_transform in tta_transforms:
                # Process each image individually with the current TTA transform
                augmented_images = torch.stack([tta_transform(img) for img in images])
                augmented_images = augmented_images.to(device)
                outputs = model(augmented_images)
                probs = F.softmax(outputs, dim=1)
                batch_preds.append(probs.cpu())
            
            # Average the predictions from all TTA versions
            avg_preds = torch.mean(torch.stack(batch_preds), dim=0)
            _, final_preds = avg_preds.max(1)
            
            # Append the predictions for each image in the batch
            for i in range(len(final_preds)):
                predictions.append({"ID": ids[i], "Labels": final_preds[i].item()})

    # Convert predictions to a DataFrame and save as CSV
    df_predictions = pd.DataFrame(predictions)
    output_csv = 'test_predictions_nearly_final_new_tta.csv'
    df_predictions.to_csv(output_csv, index=False)
    print(f"Final predictions with TTA saved to: {output_csv}")
    return df_predictions

In [12]:
# Custom ResNet Model

class SEBlock(nn.Module):
    def __init__(self, channels, reduction=16):
        super(SEBlock, self).__init__()
        
        self.fc1 = nn.Linear(channels, channels // reduction, bias=False)
        self.fc2 = nn.Linear(channels // reduction, channels, bias=False)
        self.relu = nn.ReLU(inplace=True)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        out = F.adaptive_avg_pool2d(x, 1).view(x.size(0), -1)
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.sigmoid(out).view(x.size(0), x.size(1), 1, 1)
        return x * out

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion * planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion * planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion * planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64  

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)

        self.seblock = SEBlock(channels=self.in_planes)
        
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1) 
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.avg_pool = nn.AvgPool2d(8)  
        self.fc = nn.Linear(256 * block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.seblock(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [13]:
def EfficientResNet():
    return ResNet(BasicBlock, [7, 4, 3]) 

# Define Training and Evaluation
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = EfficientResNet().to(device)

best_model_path = "/kaggle/input/retesting_previous_with_contrastedge-in-tta/pytorch/default/1/best_model_finetuned.pth"

In [14]:
# Generate prediction csv
final_predictions_df = generate_final_predictions_csv(model, best_model_path, device)
final_predictions_df

<ipython-input-11-bada4cfdacda>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(best_model_path, map_location=device))


Final predictions with TTA saved to: test_predictions_nearly_final_new_tta.csv


,ID,Labels
0,0,6
1,1,1
2,2,8
3,3,6
4,4,9
...,...,...
9995,9995,4
9996,9996,0
9997,9997,2
9998,9998,5
